In [4]:
#!/usr/bin/env python3

import pandas as pd
import errno
import os, sys
import numpy as np

'''
Author: Enrico Ceccolini
    Given the raw data of Eurora (CORE_MEASUREMENT_FULL)
    and a time interval, it returns only the measurements
    of that period, sampled at 1 minute intervals
'''


datadir = "/datasets/eurora_data/db/"

# settings
global_start_time = pd.to_datetime('2014-03-24')
global_end_time = pd.to_datetime('2015-08-12')

#interval_comment = "Andrea"
#start_time = pd.to_datetime('2014-03-31')
#end_time = pd.to_datetime('2014-05-01')

node = '01'
infile = datadir + "CPUs/cpu" + node + ".csv"

# read dataset
whole_node_power_data = pd.read_csv(infile) # Read a comma-separated values (csv) file into DataFrame
print("there are {} jobs in the whole dataset".format(whole_node_power_data.shape[0]))

there are 7158716 jobs in the whole dataset


In [5]:
whole_node_power_data.head(1)

,01,cpu_id,timestamp,pow_cpu,pow_dram,pow_pkg
0,1,0,2014-03-24 13:56:30,2.016754,2.139374,17.738455


In [6]:
whole_node_power_data = whole_node_power_data.drop([str(node)], axis=1)

In [7]:
# drop useless columns
# whole_node_power_data = whole_node_power_data.drop(['01'], axis=1)

I notice that there are duplicated rows (same timestamp and cpu_id, quite strange).

In [8]:
# drop the duplicates rows (same timestamp, cpu_id, ...)
whole_node_power_data = whole_node_power_data.drop_duplicates()
print("data rows after drop_duplicates {}".format(whole_node_power_data.shape[0]))

data rows after drop_duplicates 7057475


drop the rows before the selected started time

In [9]:
# select data in the specified interval
#interval_data = whole_node_power_data[pd.to_datetime(whole_node_power_data['timestamp']) >= start_time]
#interval_data = interval_data[pd.to_datetime(interval_data['timestamp']) <= end_time]
#print("data row from {} to {}: {}".format(start_time, end_time, interval_data.shape[0]))

In [34]:
print(node)
infile = datadir + "CPUs/cpu" + node + ".csv"

# read dataset
whole_node_power_data = pd.read_csv(infile) # Read a comma-separated values (csv) file into DataFrame
print("data rows {}".format(whole_node_power_data.shape))

# drop useless columns
whole_node_power_data = whole_node_power_data.drop([str(node)], axis=1)

# drop the duplicates rows (same timestamp, cpu_id, ...)
whole_node_power_data = whole_node_power_data.drop_duplicates() 
print("data rows after drop_duplicates {}".format(whole_node_power_data.shape[0]))

01
data rows (7158716, 6)
data rows after drop_duplicates 7057475


In [35]:
# select data in the specified interval
#interval_data = whole_node_power_data.loc[(pd.to_datetime(whole_node_power_data['timestamp']) >= start_time) & (pd.to_datetime(whole_node_power_data['timestamp']) <= end_time)]
#print("data row from {} to {}: {}".format(start_time, end_time, interval_data.shape[0]))
# no more interval

whole_node_power_data['pow_tot'] = whole_node_power_data['pow_cpu'] + whole_node_power_data['pow_dram'] +  whole_node_power_data['pow_pkg']
whole_node_power_data = whole_node_power_data.drop(['pow_dram', 'pow_pkg', 'pow_cpu'], axis=1)
whole_node_power_data.head(1)


,cpu_id,timestamp,pow_tot
0,0,2014-03-24 13:56:30,21.894583


In [36]:
interval_data = whole_node_power_data

create two different dataFrame. One for cpu0 and one for cpu1

In [37]:
# split the data referring to the cpu0 and cpu0
cpu0 = interval_data[interval_data['cpu_id'] == 0]
cpu1 = interval_data[interval_data['cpu_id'] == 1]
print("data row after split: {}".format(cpu0.shape[0]))
print("data row after split: {}".format(cpu1.shape[0]))

data row after split: 3528775
data row after split: 3528700


In [38]:
# drop the duplicates rows 
cpu0 = cpu0.drop_duplicates(subset=['timestamp'])
cpu1 = cpu1.drop_duplicates(subset=['timestamp'])
print("data row after split: {}".format(cpu0.shape[0]))
print("data row after split: {}".format(cpu1.shape[0]))

data row after split: 3440438
data row after split: 3440276


In [39]:
if cpu0.shape[0] < cpu1.shape[0]:
    cpu1 = cpu1[cpu1.timestamp >= cpu0['timestamp'].iloc[0]]
if cpu1.shape[0] < cpu0.shape[0]:
    cpu0 = cpu0[cpu0.timestamp >= cpu1['timestamp'].iloc[0]]
print("data row after split: {}".format(cpu0.shape[0]))
print("data row after split: {}".format(cpu1.shape[0]))

data row after split: 3440438
data row after split: 3440276


In [40]:
cpu1.head(1)

,cpu_id,timestamp,pow_tot
12683,1,2014-03-24 13:56:30,22.261107


convert the timestamp column to the pd datetime format

In [41]:
# convert the timestamp column to the Pandas timestamp format
cpu0.timestamp = pd.to_datetime(cpu0.timestamp)
cpu1.timestamp = pd.to_datetime(cpu1.timestamp)
cpu0.head(1)

,cpu_id,timestamp,pow_tot
0,0,2014-03-24 13:56:30,21.894583


In [24]:
cpu1.head(1)

,cpu_id,timestamp,pow_cpu
12683,1,2014-03-24 13:56:30,2.377958


In [42]:
#cpu0 = cpu0.append({'timestamp' : global_start_time} , ignore_index=True)
#cpu0.tail(1)

In [43]:
cpu0 = cpu0.set_index(["timestamp"])
cpu1 = cpu1.set_index(["timestamp"])
cpu0.head(5)

,cpu_id,pow_tot
timestamp,,
2014-03-24 13:56:30,0,21.894583
2014-03-24 13:56:35,0,21.722541
2014-03-24 13:56:40,0,22.317117
2014-03-24 13:56:45,0,22.040542
2014-03-24 13:56:50,0,22.841408


In [44]:
# resample data from 5 sec to 1 min, using the mean for the new values 
cpu0_5sec = cpu0.resample('5S').mean()
print("data rows {}".format(cpu0_5sec.shape[0]))

data rows 6742638


In [45]:
cpu0_5sec.head()

,cpu_id,pow_tot
timestamp,,
2014-03-24 13:56:30,0.0,21.894583
2014-03-24 13:56:35,0.0,21.722541
2014-03-24 13:56:40,0.0,22.317117
2014-03-24 13:56:45,0.0,22.040542
2014-03-24 13:56:50,0.0,22.841408


In [46]:
cpu1_5sec = cpu1.resample('5S').mean()
print("data rows {}".format(cpu1_5sec.shape[0]))

data rows 6742638


In [47]:
cpu0_5sec = cpu0_5sec.drop(['cpu_id'], axis=1)
cpu1_5sec = cpu1_5sec.drop(['cpu_id'], axis=1)

In [48]:
cpu0_5sec.head(1)

,pow_tot
timestamp,
2014-03-24 13:56:30,21.894583


In [49]:
first_date = cpu0_5sec.index[0]
last_date = cpu0_5sec.index[cpu0_5sec.shape[0]-1]

In [51]:
index_before = pd.date_range(start=global_start_time, end=first_date - np.timedelta64(5, 's'), freq='5S')
index_before

DatetimeIndex(['2014-03-24 00:00:00', '2014-03-24 00:00:05',
               '2014-03-24 00:00:10', '2014-03-24 00:00:15',
               '2014-03-24 00:00:20', '2014-03-24 00:00:25',
               '2014-03-24 00:00:30', '2014-03-24 00:00:35',
               '2014-03-24 00:00:40', '2014-03-24 00:00:45',
               ...
               '2014-03-24 13:55:40', '2014-03-24 13:55:45',
               '2014-03-24 13:55:50', '2014-03-24 13:55:55',
               '2014-03-24 13:56:00', '2014-03-24 13:56:05',
               '2014-03-24 13:56:10', '2014-03-24 13:56:15',
               '2014-03-24 13:56:20', '2014-03-24 13:56:25'],
              dtype='datetime64[ns]', length=10038, freq='5S')

In [52]:
index_after = pd.date_range(start=last_date + np.timedelta64(5, 's'), end=global_end_time, freq='5s')
index_after

DatetimeIndex(['2015-04-18 18:43:00', '2015-04-18 18:43:05',
               '2015-04-18 18:43:10', '2015-04-18 18:43:15',
               '2015-04-18 18:43:20', '2015-04-18 18:43:25',
               '2015-04-18 18:43:30', '2015-04-18 18:43:35',
               '2015-04-18 18:43:40', '2015-04-18 18:43:45',
               ...
               '2015-08-11 23:59:15', '2015-08-11 23:59:20',
               '2015-08-11 23:59:25', '2015-08-11 23:59:30',
               '2015-08-11 23:59:35', '2015-08-11 23:59:40',
               '2015-08-11 23:59:45', '2015-08-11 23:59:50',
               '2015-08-11 23:59:55', '2015-08-12 00:00:00'],
              dtype='datetime64[ns]', length=1991005, freq='5S')

In [53]:
padding_before = pd.DataFrame(index=index_before, columns=['pow_tot'])
padding_before.index.name = 'timestamp'
#padding_before

In [54]:
padding_after = pd.DataFrame(index=index_after, columns=['pow_tot'])
padding_after.index.name = 'timestamp'
# padding_after

In [55]:
# convert the timestamp column to the Pandas timestamp format
#cpu0.timestamp = pd.to_datetime(cpu0.timestamp)
#cpu1.timestamp = pd.to_datetime(cpu1.timestamp)

In [56]:
cpu0_5sec = pd.concat([padding_before, cpu0_5sec])
cpu1_5sec = pd.concat([padding_before, cpu1_5sec])
cpu0_5sec = pd.concat([cpu0_5sec, padding_after])
cpu1_5sec = pd.concat([cpu1_5sec, padding_after])

In [57]:
cpu0_5sec

,pow_tot
timestamp,
2014-03-24 00:00:00,NaN
2014-03-24 00:00:05,NaN
2014-03-24 00:00:10,NaN
2014-03-24 00:00:15,NaN
2014-03-24 00:00:20,NaN
2014-03-24 00:00:25,NaN
2014-03-24 00:00:30,NaN
2014-03-24 00:00:35,NaN
2014-03-24 00:00:40,NaN


In [58]:
cpu0_5sec = cpu0_5sec.rename(index=str, columns={"pow_tot": "pow_tot_0"})
cpu1_5sec = cpu1_5sec.rename(index=str, columns={"pow_tot": "pow_tot_1"})
# cpu0_1min.head(1)

In [59]:
# concat
node_5sec = pd.concat([cpu0_5sec, cpu1_5sec], axis=1)

In [60]:
node_5sec.head(1)

,pow_tot_0,pow_tot_1
timestamp,,
2014-03-24 00:00:00,NaN,NaN


In [61]:
node_5sec['pow_tot'] = node_5sec['pow_tot_0'] + node_5sec['pow_tot_1']
node_5sec = node_5sec.drop(['pow_tot_0', 'pow_tot_1'], axis=1)
node_5sec.head(1)


,pow_tot
timestamp,
2014-03-24 00:00:00,NaN


In [77]:
#pow_tot_1 = cpu0_1min['pow_cpu_0'] + cpu0_1min['pow_dram_0'] + cpu0_1min['pow_pkg_0']
#pow_tot_2 = cpu1_1min['pow_cpu_1'] + cpu1_1min['pow_dram_1'] + cpu1_1min['pow_pkg_1']
#print("first: {}, second: {}".format(pow_tot_1.shape, pow_tot_2.shape))

first: (728641,), second: (728641,)


In [63]:
# write
interval_comment = "WholeData"
outfile = datadir + 'CPUs/' + interval_comment + "/node" + node + "_5sec_" + interval_comment + ".csv"
node_5sec.to_csv(outfile)

In [64]:
infile = datadir + 'CPUs/' + interval_comment + "/node" + node + "_5sec_" + interval_comment + ".csv"

# read dataset
whole_node_power_data = pd.read_csv(infile)

whole_node_power_data = whole_node_power_data.drop(['timestamp'], axis=1)
outfile = datadir + 'CPUs/' + interval_comment + "/node" + node + "_5sec_noTimestamp_" + interval_comment + ".csv"
whole_node_power_data.to_csv(outfile)